Used for finding the correct pixel coordinates in an image:

In [ ]:
from PIL import Image

img = Image.open('c9_new.jpg') #open the image

pix_x, pix_y = 1450, 920  #set the image coordinates

red = (255, 0, 0) #red pixel so it can be easily identified 

#Modify a x by x area around the pixel
for i in range(pix_x, pix_x + 1):
    for j in range(pix_y, pix_y + 1):
        img.putpixel((i, j), red)

img.show()

Used to transform Northing and Easting to Lat-Long:

In [ ]:
import pyproj

#RD New = (Amersfoort) CRS
proj_rd_new = pyproj.Proj(init="epsg:28992")

#WGS84 CRS (used for lat/lon)
proj_wgs84 = pyproj.Proj(init="epsg:4326")

#List of points GCP locations EAST_NORTH
points_rd = [
    ("gcp0901", 72560.689, 451585.03),
    ("gcp0902", 72430.207, 451376.846),
    ("gcp0903", 72404.804, 451496.345),
    ("gcp0904", 72503.475, 451753.97),
    ("gcp0905", 72518.993, 451923.906),
    ("gcp0906", 72491.774, 451918.553),
    ("gcp0907", 72445.762, 451871.248),
    ("gcp0908", 72471.915, 451936.434),
    ("gcp0909", 72517.762, 451983.216),
    ("gcp0910", 72476.739, 451991.085),
    ("cam9",	72502.827, 452071.438)
]

#Convert RD New to WGS84 (lat/lon)
points_wgs84 = [(point[0], *pyproj.transform(proj_rd_new, proj_wgs84, point[1], point[2])) for point in points_rd]
print(points_wgs84)


c:\Users\Thomas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\Thomas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Thomas\AppData\Local\Temp\ipykernel_17060\475230796.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/g

[('gcp0901', 4.185440605113732, 52.046443983534395),
 ('gcp0902', 4.183589193882612, 52.04455372046808),
 ('gcp0903', 4.1831900966069675, 52.04562381088798),
 ('gcp0904', 4.184565960094108, 52.047953646575024),
 ('gcp0905', 4.1847511173419285, 52.049483081906665),
 ('gcp0906', 4.184355680384403, 52.04943092589389),
 ('gcp0907', 4.183696463504096, 52.048998969112695),
 ('gcp0908', 4.184061908132926, 52.04958865626547),
 ('gcp0909', 4.184718858459477, 52.0500158865175),
 ('gcp0910', 4.184119021817647, 52.05008049429814),
 ('cam9', 4.184479871385774, 52.05080646824063)]

Used for finding the distance of a point on the ground to the camera to estimate the tilt of the camera:

In [ ]:
from math import radians, sin, cos, sqrt, atan2
import numpy as np
# Define the Haversine formula to calculate distance between two points on the Earth
def haversine(lat1, lon1, lat2, lon2):
    
    R = 6371000  #radius of Earth in m

    #convert lat and lon from deg to rad
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    #Find the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    #Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance #in meters

#Coordinates of cam9 and gcp0906
lat_cam9, lon_cam9 = 52.05080646824063, 4.184479871385774
lat_gcp0906, lon_gcp0906 = 52.04943092589389, 4.184355680384403

#Calculate the distance
distance_meters = haversine(lat_cam9, lon_cam9, lat_gcp0906, lon_gcp0906)
distance_meters

correction = 2 #as a point slightly closer was used (gcp 6)
distance = distance_meters + correction
cam_height = 15

tilt = np.tanh(cam_height/distance)
tilt_deg = np.degrees(tilt)
print("Tilt in Radians",tilt)
print("Tilt in Degrees",tilt_deg)






Used to estimate the azimuth direction of the camera:

In [ ]:
from math import degrees, atan2
import numpy as np
#Function to calculate azimuth between two points
def calculate_azimuth(lat1, lon1, lat2, lon2):
    dlon = radians(lon2 - lon1)
    lat1, lat2 = radians(lat1), radians(lat2)
    
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    
    initial_bearing = atan2(x, y)
    initial_bearing = degrees(initial_bearing)
    
    #Normalize the bearing to 0-360 degrees
    azimuth = (initial_bearing + 360) % 360
    return azimuth

#find the azimuth between cam9 and gcp0906
azimuth_angle = calculate_azimuth(lat_cam9, lon_cam9, lat_gcp0906, lon_gcp0906)
azimuth_angle

#find the counterclockwise angle by subtracting the azimuth from 360
ccw_angle = (360 - azimuth_angle) % 360
ccw_angle
print("Azimuth in Radians", np.radians(ccw_angle))
print("Azimuth in Degrees",ccw_angle)
print("The camera focal lenght is 12.5mm")

The creation of the M matrix:

In [ ]:
phi = np.radians(-5.53)  #in degrees
k = np.radians(176.821)  #in degrees
w = np.radians(0)

M = np.array([
    [np.cos(phi) * np.cos(k), np.cos(w) * np.sin(k) + np.sin(w) * np.sin(phi) * np.cos(k), np.sin(w) * np.sin(k) - np.cos(w) * np.sin(phi) * np.cos(k)],
    [-np.cos(k) * np.sin(k), np.cos(w) * np.cos(k) - np.sin(w) * np.sin(phi) * np.sin(k), np.sin(w) * np.cos(k) + np.cos(w) * np.sin(phi) * np.sin(k)],
    [np.sin(phi), -np.sin(w) * np.cos(phi), np.cos(w) * np.cos(phi)]
])

print(M)

Using the location from our fieldwork (the bonus GCP) in lat/lon and translating to east-northing:

In [ ]:
#set up WGS84 ellipsoid constants
a = 6378137.0  # emi-major axis in meters
f = 1 / 298.257223563  # lattening
e2 = 2 * f - f**2  #square of eccentricity

#ECEF coordinates from rinex file
X = 3920198.6566
Y = 286789.8723
Z = 5006207.1581

#longitude
lon = np.arctan2(Y, X)

#latitude
p = np.sqrt(X**2 + Y**2)
lat = np.arctan2(Z, p * (1 - e2))  #initial guess

for _ in range(5):
    N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
    h = p / np.cos(lat) - N
    lat = np.arctan2(Z, p * (1 - e2 * N / (N + h)))

#find final latitude and height
N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
h = p / np.cos(lat) - N

#convert to degrees
lat_deg = np.degrees(lat)
lon_deg = np.degrees(lon)

lat_deg, lon_deg


(52.049183608337685, 4.184132079648719, 46.191717863082886)